# Fitting the empirical model to the entire dataset and only the training set to obtain the fitting accuracy and fitted empirical parameters

In [1]:
import numpy as np
from scipy.optimize import least_squares
from lmfit import minimize, Parameters

import pandas as pd

import matplotlib.pyplot as plt


Load interpolated capacity data and create an additional subset of data for only training cells

In [3]:
data = pd.read_csv("../Data_preprocessing/NMC_data_interp_clean.csv")
# Find the locations for training data
training_cells = pd.read_csv("../Data_preprocessing/training.csv",header=None).to_numpy(dtype=str).reshape(-1,).tolist()
mask = np.isin(data.cellID,training_cells)
data_train = data[mask]

Define the bilevel optimization algorithm

In [3]:
def optimize_bilevel(x, y, cellNums, modelEq, p0_gbl, p0_lcl,lbs,ubs):
    # Scope p_lcl globally as in the original MATLAB code
    unique_cells = np.unique(cellNums)
    p_lcl = np.ones((len(unique_cells), len(p0_lcl))) * p0_lcl
    y_fit_gbl = np.zeros_like(y)

    # Define the global equation for fitting
    def global_eq(p_gbl, x):
        y_fit = np.zeros(x.shape[0])
        iter_ = 0
        for cellNum in unique_cells:
            mask = (cellNums == cellNum)
            x_lcl = x[mask]
            y_lcl = y[mask]
            
            # Local equation based on the model
            def local_eq(params, x_lcl):
                p_lcl_iter = params.valuesdict()
                return modelEq(p_gbl, p_lcl_iter, x_lcl)

            # Using lmfit for local optimization
            params = Parameters()
            for i in range(len(p0_lcl)):
                params.add(f'p_lcl_{i}', value=p_lcl[iter_, i], min=lbs[i], max=ubs[i])
            # Minimize residuals: difference between actual y and model prediction            
            result = minimize(lambda params: y_lcl - local_eq(params, x_lcl),
                            params, 
                            method='leastsq')

            # Update p_lcl with the optimized result
            p_lcl[iter_] = np.array([result.params[f'p_lcl_{i}'].value for i in range(len(p0_lcl))])
            # Compute the fitted y values for this cell
            y_fit[mask] = local_eq(result.params, x_lcl)
            iter_ += 1

        # Update global fitted y
        nonlocal y_fit_gbl
        y_fit_gbl = y_fit
        return y - y_fit  # Return residuals (y_true - y_predicted)

    # Optimize the global equation
    res_global = least_squares(global_eq, p0_gbl, args=(x,), bounds=(0.4, 0.6)) # Bounds for the global parameter is (0.4,0.6)
    p_gbl = res_global.x
    y_fit = y_fit_gbl # Fitted global y values after optimization
    R = y - y_fit # Residuals (y_true - y_predicted)

    # Fit metrics
    MSD = np.sum(R) / len(R)
    MAE = np.mean(np.abs(R))
    percentError = np.divide(R, y, out=np.zeros_like(R), where=y!=0)
    MAPE = np.mean(np.abs(percentError[~np.isnan(percentError) & ~np.isinf(percentError)]))
    R2 = 1 - np.sum(R**2) / np.sum((y - np.mean(y))**2)
    DOF = len(y) - (len(p_gbl) + np.size(p_lcl))
    R2adj = 1 - (1 - R2) * (len(y) - 1) / DOF
    MSE = np.sum(R**2) / DOF
    RMSE = np.sqrt(MSE)

    # Assemble the result as a dictionary
    fitResult = {
        'x': x,
        'y': y,
        'y_fit': y_fit,
        'R': R,
        'p_gbl': p_gbl,
        'p_lcl': p_lcl,
        'cellID_p_lcl': unique_cells,
        'MAE': MAE,
        'MAPE': MAPE,
        'R2': R2,
        'R2adj': R2adj,
        'MSE': MSE,
        'RMSE': RMSE,
        'MSD': MSD,
    }

    return fitResult

Define the empirical model for fitting and bounds for local parameters

In [4]:
def empirical_model(p_gbl, p_lcl, x):
    return 1 - p_lcl['p_lcl_0']*x**p_gbl[0] - 1/(1+np.exp((p_lcl['p_lcl_1']-x)/(p_lcl['p_lcl_2'])))

# Lower bounds for three local parameters
lbs = [1e-6,1e2,1e1] # Avoid absolute 0 leading to numerical errors
ubs = [1,2000,500]

# Initial guess for p_gbl and p_lcl
p0_gbl = np.array([0.5])
p0_lcl = np.array([2e-3,200,50])

# Fit the entire dataset

In [5]:
x_all = np.abs(data.Ah_throughput.to_numpy())
y_all = np.abs(data.qdis.to_numpy())
cell_num_all = data.cellID

fit_result_all = optimize_bilevel(x_all,y_all,cell_num_all,empirical_model,p0_gbl,p0_lcl,lbs,ubs)
print(f"Fitting MAE for the entire dataset: {fit_result_all['MAE']*100:.3f}%")

Fitting MAE for the entire dataset: 0.566%


# Fit only the training set

In [6]:
x_train = np.abs(data_train.Ah_throughput.to_numpy())
y_train = np.abs(data_train.qdis.to_numpy())
cell_num_train = data_train.cellID

fit_result_train = optimize_bilevel(x_train,y_train,cell_num_train,empirical_model,p0_gbl,p0_lcl,lbs,ubs)
print(f"Fitting MAE for the training set: {fit_result_train['MAE']*100:.3f}%")

Fitting MAE for the training set: 0.634%


# Save results

In [ ]:
# Save fitted parameters for model training and evaluation
b_fitted_all = pd.DataFrame(np.hstack([fit_result_all['cellID_p_lcl'].reshape(-1,1),fit_result_all['p_lcl']]),columns=['Cell','b1','b2','b3'])
b_fitted_all.to_csv('Empirical_parameters_all_py.csv')
np.savetxt('Empirical_parameters_global_all_py.csv',fit_result_all['p_gbl'],delimiter=",")
b_fitted_train = pd.DataFrame(np.hstack([fit_result_train['cellID_p_lcl'].reshape(-1,1),fit_result_train['p_lcl']]),columns=['Cell','b1','b2','b3'])
b_fitted_train.to_csv('Empirical_parameters_train_py.csv')
np.savetxt('Empirical_parameters_global_train_py.csv',fit_result_train['p_gbl'],delimiter=",")